In [1]:
import numpy as np

# Define Pauli matrices (2x2)
X = np.array([[0, 1],
              [1, 0]], dtype=complex)
Y = np.array([[0, -1j],
              [1j, 0]], dtype=complex)
Z = np.array([[1, 0],
              [0, -1]], dtype=complex)

# Construct tensor product matrices and multiply by i
iXX = 1j * np.kron(X, X)
iYY = 1j * np.kron(Y, Y)
iZZ = 1j * np.kron(Z, Z)

# Function to compute the Hilbert-Schmidt inner product <A, B> = Tr(A^dagger B)
def hs_inner(A, B):
    return np.trace(np.conjugate(A.T) @ B)

# Compute the inner products
print("<iXX, iXX> =", hs_inner(iXX, iXX))
print("<iXX, iYY> =", hs_inner(iXX, iYY))
print("<iXX, iZZ> =", hs_inner(iXX, iZZ))
print("<iYY, iYY> =", hs_inner(iYY, iYY))
print("<iYY, iZZ> =", hs_inner(iYY, iZZ))
print("<iZZ, iZZ> =", hs_inner(iZZ, iZZ))


<iXX, iXX> = (4+0j)
<iXX, iYY> = 0j
<iXX, iZZ> = 0j
<iYY, iYY> = (4+0j)
<iYY, iZZ> = 0j
<iZZ, iZZ> = (4+0j)


In [2]:
# test svd 
import numpy as np
# Create a random matrix A (for example, 4x3)
A = np.random.randn(4, 3)

# Compute the singular value decomposition of A
U, S, Vh = np.linalg.svd(A, full_matrices=False)

# Verify the decomposition: A should be equal to U * diag(S) * Vh
Sigma = np.diag(S)
A_reconstructed = U @ Sigma @ Vh

print("Original Matrix A:\n", A)
print("Reconstructed Matrix A_reconstructed:\n", A_reconstructed)


Original Matrix A:
 [[ 0.42355798 -0.25100407 -0.38708598]
 [-1.27076764  0.68370485  1.68913668]
 [-1.38849281  1.68313778 -0.64058155]
 [ 0.39789637  2.05612316  0.35615391]]
Reconstructed Matrix A_reconstructed:
 [[ 0.42355798 -0.25100407 -0.38708598]
 [-1.27076764  0.68370485  1.68913668]
 [-1.38849281  1.68313778 -0.64058155]
 [ 0.39789637  2.05612316  0.35615391]]


In [1]:
import numpy as np

# Suppose N is given (for example, N=4)
N = 4

# Create a random tensor T with shape (2,)*2N
T = np.random.rand(*(2 for _ in range(2 * N)))

# Step 1: Permute axes so that index 0 and index N become the first two indices.
# Original order: [0, 1, 2, ..., N-1, N, N+1, ..., 2N-1]
# Desired order: [0, N, 1, 2, ..., N-1, N+1, ..., 2N-1]
axes_order = [0, N] + [i for i in range(1, 2 * N) if i != N]
T_perm = T.transpose(axes_order)

# Step 2: Reshape the tensor into a matrix.
# The first two dimensions are grouped together: new row dimension = 2*2 = 4.
# The remaining dimensions form the column dimension: 2**(2N - 2).
row_dim = 2 * 2
col_dim = 2 ** (2 * N - 2)
T_matrix = T_perm.reshape(row_dim, col_dim)

# Step 3: Perform the SVD on the reshaped matrix.
U, S, Vh = np.linalg.svd(T_matrix, full_matrices=False)

# Now, U contains the left part of the SVD.
# To interpret U as the first site tensor of an MPO, you might want to reshape it.
# For example, if you expect the site tensor to have shape (physical_in, physical_out, bond_dim),
# you could reshape U as needed. Here we reshape U to (2, 2, -1) assuming that grouping the two physical indices is desired.
first_site_tensor = U.reshape(2, 2, -1)

print("Shape of U (after SVD):", U.shape)
print("Shape of first site tensor:", first_site_tensor.shape)

Shape of U (after SVD): (4, 4)
Shape of first site tensor: (2, 2, 4)


In [3]:
import numpy as np
from scipy.linalg import expm, norm
from qiskit.synthesis import TwoQubitWeylDecomposition

# Helper function: Generate a random unitary matrix using QR decomposition.
def random_unitary(n):
    X = np.random.randn(n, n) + 1j * np.random.randn(n, n)
    Q, R = np.linalg.qr(X)
    # Normalize phases so that the diagonal of R has positive entries
    d = np.diag(R)
    Q = Q * np.exp(-1j * np.angle(d))
    return Q

# Define the 2x2 Pauli matrices.
X = np.array([[0, 1],
              [1, 0]])
Y = np.array([[0, -1j],
              [1j, 0]])
Z = np.array([[1, 0],
              [0, -1]])

# Create the two-qubit Pauli operators (Kronecker products).
XX = np.kron(X, X)
YY = np.kron(Y, Y)
ZZ = np.kron(Z, Z)

# Generate a random 4x4 unitary.
U = random_unitary(4)

# Decompose U using TwoQubitWeylDecomposition.
decomp = TwoQubitWeylDecomposition(U)

# Retrieve the decomposition parameters and local unitaries.
a = decomp.a
b = decomp.b
c = decomp.c
K1l = decomp.K1l
K1r = decomp.K1r
K2l = decomp.K2l
K2r = decomp.K2r
global_phase = decomp.global_phase  # This is an overall phase factor

# Compute the nonlocal part: exp[i*(a*XX + b*YY + c*ZZ)].
nonlocal_op = expm(1j * (a * XX + b * YY + c * ZZ))

# Reconstruct U: Note that the full decomposition is given up to a global phase.
U_reconstructed = np.kron(K1l, K1r) @ nonlocal_op @ np.kron(K2l, K2r)

# Adjust the reconstructed unitary by the extracted global phase.
U_reconstructed *= np.exp(1j * global_phase)

# Check the reconstruction: two unitaries are equivalent if they differ only by a global phase.
# One method is to compare the matrices up to a phase factor.
def unitary_equal_up_to_global_phase(U1, U2, atol=1e-10):
    # Compute relative phase using the (0,0) element (if nonzero)
    phase = U1[0,0] / U2[0,0] if np.abs(U2[0,0]) > atol else 1.0
    return np.allclose(U1, phase * U2, atol=atol)

reconstruction_ok = unitary_equal_up_to_global_phase(U, U_reconstructed)

# Print the results.
print("Random 4x4 unitary U:")
print(U)
print("\nDecomposition parameters (a, b, c):")
print(a, b, c)
print("\nGlobal phase (in radians):")
print(global_phase)
print("\nReconstruction check (True means success):", reconstruction_ok)


Random 4x4 unitary U:
[[ 0.33702837-0.08624403j  0.29722202-0.41342211j  0.09996544+0.41734901j
  -0.4920394 -0.43981703j]
 [ 0.19624253-0.13917188j -0.32304204-0.39499876j -0.74213455-0.34467132j
  -0.07630969-0.07971634j]
 [-0.24597709-0.09034285j -0.19974883+0.48194847j  0.05623441-0.35709071j
  -0.48837538-0.53849134j]
 [-0.30712198+0.81123574j -0.34761775-0.2955487j   0.0435592 +0.11700945j
  -0.12354419-0.09237827j]]

Decomposition parameters (a, b, c):
0.5443742687290278 0.36197534561046224 0.024278736087296693

Global phase (in radians):
-0.2726686981455577

Reconstruction check (True means success): True
